In [1]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
from massimal import hyspec_io, image_render, preprocess
from tqdm import tqdm

In [2]:
# Paths
hyspec_base = pathlib.Path('/home/mha114/data/massimal/vega_sola/hyperspec/images/2a_rrs/')
water_mask_base = pathlib.Path('/home/mha114/data/massimal/vega_sola/hyperspec/watermask/')
# rgb_dir = pathlib.Path('/home/mha114/data/massimal/vega_sola/hyperspec/rgb/rgb_log_v1/')
rgb_dir = pathlib.Path('/home/mha114/data/massimal/vega_sola/hyperspec/rgb/rgb_log_v2/')
subfolders = ['Coast1','Coast2','Coast3','Sound1','Sound2','Sound3a','Sound3b']

# Create RGB subfolder if it doesn't exist
rgb_dir.mkdir(exist_ok=True)

In [3]:
# Parameters
rgb_mod_wl = [640,570,500]
rgb_ind = (121, 88, 54)
rgb_stretch_limits = np.array([[0.0035, 0.0110, 0.01],   
                               [0.0200, 0.0918, 0.07]]).T
minimum_reflectance = 0.001

In [4]:
# Loop through all folders and images and create percentile stretched RGB image
for subfolder in subfolders:
    print(f'Processing subfolder {subfolder}')
    
    # Construct folder paths for current subfolder
    hyspec_folder = hyspec_base / subfolder
    water_mask_folder = water_mask_base / subfolder

    # Loop through all images which have water masks (some or all of image has water)
    # Inpaint missing pixels and mask out non-water areas, save RGG images
    image_stems = [pathlib.Path(image_path).stem for image_path in sorted(water_mask_folder.glob('*.png'))]
    
    for image_stem in tqdm(image_stems,leave=True):
        (hyspec_im,wl,_,_) = hyspec_io.load_envi_image(str(hyspec_folder/image_stem)+'.hdr')
        rgb_im = hyspec_im[:,:,rgb_ind]
        water_mask = np.expand_dims(skimage.io.imread(str(water_mask_folder/image_stem)+'.png'), 2)
        zeros_mask = np.all(rgb_im==0,axis=2,keepdims=True)
        rgb_im_inpainted = preprocess.inpaint_masked(rgb_im,np.squeeze(water_mask*zeros_mask))
        rgb_im_inpainted[rgb_im_inpainted<minimum_reflectance] = minimum_reflectance
        rgb_im_stretched = image_render.absolute_stretch(
            np.log10(rgb_im_inpainted),limits=np.log10(rgb_stretch_limits))
        rgb_im_out = np.uint8(rgb_im_stretched*water_mask*255)
        
        transparency_layer = np.ones(shape=(rgb_im_out.shape[0:2])+(1,),dtype=np.uint8)*255
        rgb_im_out = np.concatenate((rgb_im_out,transparency_layer),axis=2)
        # plt.imshow(rgb_im_out)
        # plt.show()
        
        skimage.io.imsave(str(rgb_dir/image_stem)+'.png',rgb_im_out, check_contrast=False)

Processing subfolder Coast1


100%|██████████| 27/27 [01:04<00:00,  2.39s/it]


Processing subfolder Coast2


100%|██████████| 27/27 [00:54<00:00,  2.00s/it]


Processing subfolder Coast3


...

100%|██████████| 30/30 [04:48<00:00,  9.62s/it]


Processing subfolder Sound3a


100%|██████████| 28/28 [01:32<00:00,  3.30s/it]


Processing subfolder Sound3b


100%|██████████| 27/27 [03:20<00:00,  7.41s/it]
